In [1]:
import Bio
import Bio.Restriction
import Bio.SeqIO
import Bio.SearchIO
import random
import matplotlib.pyplot as plt
%matplotlib inline

/s/chopin/l/grad/muggli/py3.5env/lib/python3.5/site-packages/Bio/SearchIO/__init__.py:211: BiopythonExperimentalWarning: Bio.SearchIO is an experimental submodule which may undergo significant changes prior to its future official release.
  BiopythonExperimentalWarning)


In [2]:
with open("../ecoli_ref.fa", "rU") as ref_handle:
    for ref_seq in Bio.SeqIO.parse(ref_handle, "fasta"): # iterate through the all (in this case one) seqs in the fasta file
        ecoli = ref_seq

/s/chopin/l/grad/muggli/py3.5env/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: 'U' mode is deprecated
  if __name__ == '__main__':


In [3]:
random.seed(12345)
loci = [random.randint(0,len(ecoli)) for x in range(100)]
loci.sort()

diffs = [y - x for x,y in zip(loci[:-1], loci[1:])]
min(diffs)
loci.reverse()

In [19]:
type(ecoli)

Bio.SeqRecord.SeqRecord

In [35]:
mutable_variant =variant.seq.seq.tomutable()

In [48]:
# boring work taken from http://stackoverflow.com/questions/21205836/generating-random-sequences-of-dna

def DNA(length):
    return ''.join(random.choice('CGTA') for _ in range(length))

In [64]:
# >var_1_5p_flank length:35 INFO:KMER=31
# GTCAGTTCTGTTCTTGAGTTCACTGTCGCTTCAGA
# >var_1_branch_1 length:32 kmer:31
# ATGGATGGCTCCTCTGCTCTCCGCTGCCAGAG
# >var_1_branch_2 length:32 kmer:31
# TTGGATGGCTCCTCTGCTCTCCGCTGCCAGAG
# >var_1_3p_flank length:33 kmer:31
# TGGCATCGTGCTCGCTCTGCTGGGTTGCCAATT

bubbles=open("bubbles.txt", "w")

flank_size = 40
for lno, locus in enumerate(loci):
    bubbles.write(">var_" +str(lno+1)+ "_5p_flank length:"+str(flank_size)+" INFO:KMER=31\n")
    bubbles.write(str(ecoli[locus-flank_size:locus].seq) + "\n")
    varsize = random.randint(200,500)
    vartype = random.randint(1,3)
    if vartype == 1: # insertion
        newdna = DNA(varsize)
        mutable_variant[locus:locus] = newdna
        bubbles.write(">var_" +str(lno+1)+ "_branch_1 length:"+str(varsize)+" kmer:31\n")
        bubbles.write(newdna + "\n")
        bubbles.write(">var_" +str(lno+1)+ "_branch_2 length:"+str(0)+" kmer:31\n")
        bubbles.write("\n")
        right_flank = ecoli[locus:locus+flank_size]
    elif vartype == 2: # deletion
        mutable_variant[locus:locus+varsize] = ""
        bubbles.write(">var_" +str(lno+1)+ "_branch_1 length:"+str(0)+" kmer:31\n")
        bubbles.write( "\n")
        bubbles.write(">var_" +str(lno+1)+ "_branch_2 length:"+str(varsize)+" kmer:31\n")
        bubbles.write(str(ecoli[locus:locus+varsize].seq) + "\n")
        right_flank = ecoli[locus+varsize:locus+varsize+flank_size]
    else: # substitution
        newdna = DNA(varsize)
        mutable_variant[locus:locus+varsize] = newdna
        bubbles.write(">var_" +str(lno+1)+ "_branch_1 length:"+str(varsize)+" kmer:31\n")
        bubbles.write(newdna + "\n")        
        bubbles.write(">var_" +str(lno+1)+ "_branch_2 length:"+str(varsize)+" kmer:31\n")
        bubbles.write(str(ecoli[locus:locus+varsize].seq) + "\n")
        right_flank = ecoli[locus+varsize:locus+varsize+flank_size]
    bubbles.write(">var_" +str(lno+1)+ "_3p_flank length:"+str(flank_size)+" INFO:KMER=31\n")
    bubbles.write(str(right_flank.seq) + "\n")
bubbles.close()
    

In [65]:
!head bubbles.txt

>var_1_5p_flank length:40 INFO:KMER=31
TGGCATCGTTATTGCTGGAAACCGACGCGCCGGATATGCC
>var_1_branch_1 length:268 kmer:31
GCTTCCTGCCGTGCTAGGGACCCGTTCATCAGTGAGCGAACCCCGACTGACCTCGTACAATTACCGGACGAACTGTTTACCTGGTTCCACAGTGTAAGATTCAACGACGGTAACATTTATCCGAGCGGTTGCCTCATCATTGTTGGTGAGTATCATGACATTGTTAGGAACCGAAAAGTGGACAGGAGCGACCCTCACAAAAGAACGAAGTATAGCGAATGTGTGGGATACCGGACCCCGACACGAGGTCTCCCCCGCGAGGAAAGTC
>var_1_branch_2 length:0 kmer:31

>var_1_3p_flank length:40 INFO:KMER=31
GCTCAACGGTTTTCAGGGGCAGCCTAACCGCCCGGAGCAG
>var_2_5p_flank length:40 INFO:KMER=31
GCGTCACGGTTTCCGGGGTTAAGTCCTGACCCGTCAGATC


In [66]:
newecoli = Bio.SeqRecord.SeqRecord(mutable_variant)

In [71]:
with open("ecoli_variant.fasta", "w") as output_handle:
    Bio.SeqIO.write(newecoli, output_handle, "fasta")

In [70]:
!head ecolie_variant.fasta

><unknown id> <unknown description>
AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTC
TGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAATTAAAATTTTATTGACTTAGG
TCACTAAATACTTTAACCAATATAGGCATAGCGCACAGACAGATAAAAATTACAGAGTAC
ACAACATCCATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGT
AACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAGCCCGCACCTGACAGTGCGGG
CTTTTTTTTTCGACCAAAGGTAACGAGGTAACAACCATGCGAGTGTTGAAGTTCGGCGGT
ACATCAGTGGCAAATGCAGAACGTTTTCTGCGTGTTGCCGATATTCTGGAAAGCAATGCC
AGGCAGGGGCAGGTGGCCACCGTCCTCTCTGCCCCCGCCAAAATCACCAACCACCTGGTG
GCGATGATTGAAAAAACCATTAGCGGCCAGGATGCTTTACCCAATATCAGCGATGCCGAA
